In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from os.path import exists
import sklearn.linear_model as lm

import sys
sys.path.insert(0, "../util/")
import util as util

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [2]:
amesHousing = pd.read_csv('../data/Ames_Housing_Price_Data.csv')
amesRealEstate = pd.read_csv('../data/Ames_Real_Estate_Data.csv')

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
amesHousing.head()

,Unnamed: 0,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,...,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
1,2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,...,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
2,3,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,...,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
3,4,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,...,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
4,5,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,...,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal


In [4]:
set(amesHousing["ExterQual"])

{'Ex', 'Fa', 'Gd', 'TA'}

In [5]:
set(amesHousing["Functional"])

{'Maj1', 'Maj2', 'Min1', 'Min2', 'Mod', 'Sal', 'Typ'}

In [6]:
amesRealEstate.head()

,MapRefNo,GeoRefNo,Tier,Range,Prop_Addr,ZngCdPr,ZngCdSc,ZngOLPr,ZngOLSc,ClassPr_S,...,PA-Nmbr,PA-PreD,PA-Strt,PA-StSfx,PA-PostD,PA-UnTyp,PA-UntNo,Date,Source,NmbrBRs
0,520400001,520400001,0,0,4599 GEORGE W CARVER AVE,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,4599.0,NaN,GEORGE W CARVER,AVE,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,NaN
1,520440010,520440010,0,0,4507 EVEREST AVE,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,4507.0,NaN,EVEREST,AVE,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,NaN
2,520440020,520440020,0,0,4513 EVEREST AVE,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,4513.0,NaN,EVEREST,AVE,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,NaN
3,520445001,520445001,0,0,3449 BARCELOS ST,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,3449.0,NaN,BARCELOS,ST,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,NaN
4,520445010,520445010,0,0,4506 EVEREST AVE,FS-RL,NaN,NaN,NaN,RESIDENTIAL,...,4506.0,NaN,EVEREST,AVE,NaN,NaN,NaN,10-Jul-20,Ames City Assessor,NaN


In [7]:
amesHousing['MSSubClass'] = amesHousing['MSSubClass'].apply(str)

In [8]:
amesHousing['MSSubClass'].dtype

dtype('O')

In [9]:
amesHousingCategoricalData = amesHousing.select_dtypes(include=['O'])

In [10]:
amesHousingCategoricalData["Alley"].value_counts()

Grvl    105
Pave     63
Name: Alley, dtype: int64

In [11]:
for col in amesHousingCategoricalData.columns:
    print(col)
    print(set(amesHousing[col]))
    if (amesHousing[col].isnull().values.any()):
        print(amesHousing[col].value_counts())
        print(f'{amesHousing[col].isnull().sum()} nan values')

MSSubClass
{'150', '75', '70', '20', '40', '90', '80', '30', '120', '85', '50', '60', '180', '45', '190', '160'}
MSZoning
{'A (agr)', 'RH', 'RM', 'FV', 'C (all)', 'RL', 'I (all)'}
Street
{'Pave', 'Grvl'}
Alley
{nan, 'Pave', 'Grvl'}
Grvl    105
Pave     63
Name: Alley, dtype: int64
2412 nan values
LotShape
{'IR2', 'IR3', 'IR1', 'Reg'}
LandContour
{'HLS', 'Low', 'Lvl', 'Bnk'}
Utilities
{'NoSewr', 'AllPub'}
LotConfig
{'FR3', 'CulDSac', 'Inside', 'Corner', 'FR2'}
LandSlope
{'Mod', 'Sev', 'Gtl'}
Neighborhood
{'Greens', 'Edwards', 'BrkSide', 'NWAmes', 'Blmngtn', 'SawyerW', 'Somerst', 'Gilbert', 'OldTown', 'GrnHill', 'Mitchel', 'Blueste', 'Crawfor', 'IDOTRR', 'NAmes', 'CollgCr', 'NridgHt', 'NPkVill', 'BrDale', 'Timber', 'SWISU', 'Veenker', 'NoRidge', 'Landmrk', 'MeadowV', 'StoneBr', 'Sawyer', 'ClearCr'}
Condition1
{'Norm', 'Artery', 'RRAe', 'PosN', 'RRNn', 'RRNe', 'Feedr', 'PosA', 'RRAn'}
Condition2
{'Norm', 'Artery', 'RRAe', 'PosN', 'RRNn', 'Feedr', 'PosA', 'RRAn'}
BldgType
{'TwnhsE', '2fmCo

In [12]:
for col in amesHousingCategoricalData.columns:
    if (amesHousing[col].isnull().values.any()):
        print(amesHousing[amesHousing[col]=='nan'].shape[0])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [13]:
for col in amesHousingCategoricalData.columns:
    amesHousing[col]=amesHousing[col].fillna('nan')

In [14]:
for col in amesHousingCategoricalData.columns:
    if (amesHousing[col].isnull().values.any()):
        print(amesHousing[amesHousing[col]=='nan'].shape[0])

In [15]:
pd.get_dummies(amesHousing['Utilities'],prefix='Utilites',drop_first=True)

,Utilites_NoSewr
0,0
1,0
2,0
3,0
4,0
...,...
2575,0
2576,0
2577,0
2578,0


In [16]:
a = {1:1,2:2,3:3}

In [17]:
5 in a.keys()

False

In [18]:
relatedDummiesDictionary = {}
for col in amesHousingCategoricalData.columns:
    dummyData = pd.get_dummies(amesHousing[col],prefix=col,drop_first=True)
    for dummyCol in dummyData.columns:
        relatedDummiesDictionary[str(dummyCol)] = list(dummyData.columns)
    amesHousing = pd.concat([amesHousing,dummyData],axis=1)

In [19]:
set(amesHousing.dtypes)

{dtype('uint8'), dtype('int64'), dtype('float64'), dtype('O')}

In [20]:
amesHousingNumericalData = amesHousing.select_dtypes(include=['uint8','int64','float64'])
[col for col in amesHousingNumericalData.columns if amesHousingNumericalData[col].isnull().values.any()]

['LotFrontage',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea']

In [21]:
testList = [1,2,3,10000]
print(np.std(np.array(testList)))
print(np.mean(np.array(testList)))
util.markOutliers(testList)

4329.261051264984
2501.5


AttributeError: module 'util' has no attribute 'markOutliers'

In [ ]:
print(f'{amesHousingNumericalData.shape[0]} rows before removing outliers')
amesHousingNumericalData = util.removeOutliers(amesHousingNumericalData.fillna(0.0))
print(f'{amesHousingNumericalData.shape[0]} rows after removing outliers')

In [ ]:
corr = amesHousingNumericalData.corr()
corr

In [ ]:
sns.heatmap(corr)

In [ ]:
correlationList = []
pairList = []
pValList = []
for i in range(len(amesHousingNumericalData.columns)):
    for j in range(len(amesHousingNumericalData.columns)):
        if i<j:
            iColumn = amesHousingNumericalData.columns[i]
            jColumn = amesHousingNumericalData.columns[j]
            pairList.append(iColumn+'-'+jColumn)
            corrVal,pVal = stats.spearmanr(amesHousingNumericalData[iColumn],
                                          amesHousingNumericalData[jColumn])
            correlationList.append(corrVal)
            pValList.append(pVal)

amesHousingCorreltaion = pd.DataFrame({"pair":pairList,
                                "corr":correlationList,
                                "pVal":pValList
                               },columns=["pair","corr","pVal"]).sort_values(by=["pVal","corr"],
                                                                             ascending=[True,False])
            
            

In [ ]:
amesHousingSigCorrs = amesHousingCorreltaion[(np.isnan(amesHousingCorreltaion['pVal'])==False) & (amesHousingCorreltaion['pVal'] < 0.05)]

In [ ]:
absRValues = abs(amesHousingSigCorrs['corr'])

In [ ]:
type(amesHousingSigCorrs)

In [ ]:
print(amesHousingSigCorrs.size)
amesHousingSigCorrs = amesHousingSigCorrs[abs(amesHousingSigCorrs['corr']) > absRValues.quantile()]
print(amesHousingSigCorrs.size)

In [ ]:
amesHousingSigCorrs.hist(bins=10)

In [ ]:
amesHousingSigCorrs.head()

In [ ]:
amesHousingSigCorrs['pair'].str.contains('YearBuilt')

In [ ]:
[1,2,3]+[]

In [ ]:
relatedDummiesDictionary["MasVnrType_BrkFace"]

In [ ]:
sigCorrDictionary = {}
for col in amesHousingNumericalData.columns:
    correlatedRelations = (amesHousingSigCorrs[amesHousingSigCorrs['pair'].str.contains(col)])["pair"]
    correlatedColumns = [name.replace("-","").replace(col,"") for name in correlatedRelations]
    relatedDummies = []
    for corrCol in correlatedColumns:
        if corrCol in relatedDummiesDictionary.keys():
            print(corrCol)
            relatedDummies = relatedDummies + relatedDummiesDictionary[corrCol]
    correlatedColumns = list(set(correlatedColumns + relatedDummies))
    sigCorrDictionary[col] = correlatedColumns

In [ ]:
sigCorrDictionary

In [ ]:
powers = np.linspace(1,10,10)
X = (np.random.random(1000)*1000).reshape(-1, 1)
for power in powers:
    y = X ** power
    linmodel = lm.LinearRegression()
    linmodel.fit(np.log(X),np.log(y))
    print(f'power = {power} and slope is {linmodel.coef_[0,0]}')
    
    

In [ ]:
housingTrainWithDummies = pd.read_csv('../data/housingTrainWithDummies.csv')
set(housingTrainWithDummies.dtypes)

In [ ]:
values = list(np.random.random(1000)*1000)

In [ ]:
df = pd.DataFrame({"values":values},columns = ["values"])
pd.qcut(df['values'],q=10,labels=np.linspace(0,9,10),retbins=True)

In [ ]:
df.head()

In [ ]:
try:
    somethingThatDoesntExist
except NameError:
    bonkers = "me"
    x = bonkers
print(x)

In [ ]:
arr = np.array([0,-np.inf,5]).reshape(-1,1)
np.isinf(arr).any()

In [1]:
list(1)

TypeError: 'int' object is not iterable